# Starting with error handling
 

In [ ]:
BEGIN TRY
    INSERT INTO products (product_name, stock, price)
        VALUES ('Trek Powerfly 5 - 2018', 10, 3499.99);
    SELECT 'Product inserted correctly!';
END TRY

    BEGIN CATCH
        SELECT 'An error occurred! You are in the CATCH block';   
    END CATCH

In [ ]:
# Set up the TRY block
BEGIN TRY
	# Add the constraint
	ALTER TABLE products
		ADD CONSTRAINT CHK_Stock CHECK (stock >= 0);
END TRY
# Set up the CATCH block
BEGIN CATCH
	SELECT 'An error occurred!';
END CATCH

In [ ]:
# Set up the first TRY block
BEGIN TRY
	INSERT INTO buyers (first_name, last_name, email, phone)
		VALUES ('Peter', 'Thompson', 'peterthomson@mail.com', '555000100');
END TRY
# Set up the first CATCH block
BEGIN CATCH
	SELECT 'An error occurred inserting the buyer! You are in the first CATCH block';
    # Set up the nested TRY block
    BEGIN TRY
    	INSERT INTO errors 
        	VALUES ('Error inserting a buyer');
        SELECT 'Error inserted correctly!';
	END TRY
    # Set up the nested CATCH block
    BEGIN CATCH
    	SELECT 'An error occurred inserting the error! You are in the nested CATCH block';
    END CATCH 
END CATCH

# Error anatomy and uncatchable errors
 

In [ ]:
Msg 2627, Level 14, State 1, Line 1
Violation of UNIQUE KEY constraint 'unique_name'. 
Cannot insert duplicate key in object 'dbo.products'. 
The duplicate key value is (Trek Powerfly 5 - 2018).

# What are the different parts of the error you get, from left to right?
# Error number, severity level, state, line, and message text.

In [ ]:
BEGIN TRY
	INSERT INTO products (product_name, stock, price)
		VALUES ('Sun Bicycles ElectroLite - 2017', 10, 1559.99);
END TRY
BEGIN CATCH
	SELECT 'An error occurred inserting the product!';
    BEGIN TRY
    	INSERT INTO errors
        	VALUES ('Error inserting a product');
    END TRY    
    BEGIN CATCH
    	SELECT 'An error occurred inserting the error!';
    END CATCH    
END CATCH

# Giving information about errors
 

In [ ]:
# Which of the following is true about the functions: 
#ERROR_NUMBER(), ERROR_SEVERITY(), ERROR_STATE(), ERROR_PROCEDURE(), ERROR_LINE(), and ERROR_MESSAGE()?

# These functions must be placed within the CATCH block. 
# If an error occurs within the TRY block, they return information about the error.

In [ ]:
# Set up the TRY block
BEGIN TRY  	
	SELECT 'Total: ' + SUM(price * quantity) AS total
	FROM orders  
END TRY
# Set up the CATCH block
BEGIN CATCH  
	# Show error information.
	SELECT  error_number() AS number,  
        	error_severity() AS severity_level,  
        	error_state() AS state,
        	error_line() AS line,  
        	error_message() AS message; 	
END CATCH 

In [ ]:
BEGIN TRY
    INSERT INTO products (product_name, stock, price) 
    VALUES	('Trek Powerfly 5 - 2018', 2, 3499.99),   		
    		('New Power K- 2018', 3, 1999.99)		
END TRY
# Set up the outer CATCH block
BEGIN CATCH
	SELECT 'An error occurred inserting the product!';
    # Set up the inner TRY block
    BEGIN TRY
    	# Insert the error
    	INSERT INTO error 
        	VALUES ('Error inserting a product');
    END TRY    
    # Set up the inner CATCH block
    BEGIN CATCH
    	# Show number and message error
    	SELECT 
        	error_line() AS line,	   
			error_message() AS message; 
    END CATCH    
END CATCH

# RAISERROR
 

In [ ]:
RAISERROR('You cannot apply a 50%% discount on %s number %d', 6, 1, 'product', 5);
# You cannot apply a 50% discount on product number 5

In [ ]:
DECLARE @product_id INT = 5;

IF NOT EXISTS (SELECT * FROM products WHERE product_id = @product_id)
	# Invoke RAISERROR with parameters
	RAISERROR('No product with id %d.', 11, 1, @product_id);
ELSE 
	SELECT * FROM products WHERE product_id = @product_id;

    
# Change the value
DECLARE @product_id INT = 50;

IF NOT EXISTS (SELECT * FROM products WHERE product_id = @product_id)
	RAISERROR('No product with id %d.', 11, 1, @product_id);
ELSE 
	SELECT * FROM products WHERE product_id = @product_id;

In [ ]:
BEGIN TRY
	# Change the value
    DECLARE @product_id INT = 5;	
    IF NOT EXISTS (SELECT * FROM products WHERE product_id = @product_id)
        RAISERROR('No product with id %d.', 11, 1, @product_id);
    ELSE 
        SELECT * FROM products WHERE product_id = @product_id;
END TRY
BEGIN CATCH
	SELECT ERROR_MESSAGE();
END CATCH

BEGIN TRY
	# Change the value
    DECLARE @product_id INT = 50;	
    IF NOT EXISTS (SELECT * FROM products WHERE product_id = @product_id)
        RAISERROR('No product with id %d.', 11, 1, @product_id);
    ELSE 
        SELECT * FROM products WHERE product_id = @product_id;
END TRY
BEGIN CATCH
	SELECT ERROR_MESSAGE();
END CATCH

# THROW 

In [ ]:
# The THROW statement without parameters should be placed within a CATCH block.
CREATE PROCEDURE insert_product
  @product_name VARCHAR(50),
  @stock INT,
  @price DECIMAL

AS

BEGIN TRY
	INSERT INTO products (product_name, stock, price)
		VALUES (@product_name, @stock, @price);
END TRY
# Set up the CATCH block
BEGIN CATCH
	# Insert the error and end the statement with a semicolon
    INSERT INTO errors VALUES ('Error inserting a product');
    # Re-throw the error
	THROW; 
END CATCH

In [ ]:
BEGIN TRY
	# Execute the stored procedure
	EXEC insert_product 
    	# Set the values for the parameters
    	@product_name = 'Trek Conduit+',
        @stock = 3,
        @price = 499.99;
END TRY
# Set up the CATCH block
BEGIN CATCH
	# Select the error message
	SELECT ERROR_MESSAGE();
END CATCH

In [ ]:
DECLARE @staff_id INT = 4;

IF NOT EXISTS (SELECT * FROM staff WHERE staff_id = @staff_id)
   	# Invoke the THROW statement with parameters
	THROW 50001, 'No staff member with such id', 1;
ELSE
   	SELECT * FROM staff WHERE staff_id = @staff_id

# Customizing error messages in the THROW statement
 

In [ ]:
DECLARE @first_name NVARCHAR(20) = 'Edward';

# Concat the message
DECLARE @my_message NVARCHAR(500) =
	CONCAT('There is no staff member with ', @first_name, ' as the first name.');

IF NOT EXISTS (SELECT * FROM staff WHERE first_name = @first_name)
	-- Throw the error
	THROW 50000, @my_message, 1;

In [ ]:
DECLARE @product_name AS NVARCHAR(50) = 'Trek CrossRip+ - 2018';
# Set the number of sold bikes
DECLARE @sold_bikes AS INT = 10;
DECLARE @current_stock INT;

SELECT @current_stock = stock FROM products WHERE product_name = @product_name;

DECLARE @my_message NVARCHAR(500) =
	# Customize the error message
	FORMATMESSAGE('There are not enough %s bikes. You have %d in stock.', @product_name, @current_stock);

IF (@current_stock - @sold_bikes < 0)
	# Throw the error
	THROW 50000, @my_message, 1;

In [ ]:
# Pass the variables to the stored procedure
EXEC sp_addmessage @msgnum = 50002, @severity = 16, @msgtext = 'There are not enough %s bikes. You only have %d in stock.', @lang = N'us_english';

In [ ]:
EXEC sp_addmessage @msgnum = 50002, @severity = 16, @msgtext = 'There are not enough %s bikes. You only have %d in stock.', @lang = N'us_english';
DECLARE @product_name AS NVARCHAR(50) = 'Trek CrossRip+ - 2018';
DECLARE @sold_bikes AS INT = 10;
DECLARE @current_stock INT;

SELECT @current_stock = stock FROM products WHERE product_name = @product_name;

DECLARE @my_message NVARCHAR(500) =
	# Prepare the error message
	FORMATMESSAGE(50002, @product_name, @current_stock);

IF (@current_stock - @sold_bikes < 0)
	# Throw the error
	THROW 50000, @my_message, 1;

In [ ]:
EXEC sp_addmessage @msgnum = 50002, @severity = 16, @msgtext = 'There are not enough %s bikes. You only have %d in stock.', @lang = N'us_english';
DECLARE @product_name AS NVARCHAR(50) = 'Trek CrossRip+ - 2018';

# Change the value
DECLARE @sold_bikes AS INT = 10;
DECLARE @current_stock INT;

SELECT @current_stock = stock FROM products WHERE product_name = @product_name;

DECLARE @my_message NVARCHAR(500) =
	FORMATMESSAGE(50002, @product_name, @current_stock);

IF (@current_stock - @sold_bikes < 0)
	THROW 50000, @my_message, 1;

# Transactions 

In [ ]:
# The BEGIN TRAN|TRANSACTION statement marks the starting point of a transaction.
# The COMMIT TRAN|TRANSACTION statement marks the end of a successful transaction.
# The ROLLBACK TRAN|TRANSACTION statement reverts a transaction to the beginning or a savepoint inside the transaction.

In [ ]:
BEGIN TRY  
	BEGIN TRAN; 
		UPDATE accounts SET current_balance = current_balance - 100 WHERE account_id = 1;
		INSERT INTO transactions VALUES (1, -100, GETDATE());
        
		UPDATE accounts SET current_balance = current_balance + 100 WHERE account_id = 5;
		INSERT INTO transactions VALUES (5, 100, GETDATE());
	COMMIT TRAN;     
END TRY
BEGIN CATCH  
	ROLLBACK TRAN;
END CATCH;

In [ ]:
BEGIN TRY  
	# Begin the transaction
	BEGIN TRAN;
		UPDATE accounts SET current_balance = current_balance - 100 WHERE account_id = 1;
		INSERT INTO transactions VALUES (1, -100, GETDATE());
        
		UPDATE accounts SET current_balance = current_balance + 100 WHERE account_id = 5;
        # Correct it
		INSERT INTO transactions VALUES (500, 100, GETDATE());
    # Commit the transaction
	COMMIT TRAN;    
END TRY
BEGIN CATCH  
	SELECT 'Rolling back the transaction';
    # Rollback the transaction
	ROLLBACK TRAN;
END CATCH

In [ ]:
# Begin the transaction
BEGIN TRAN; 
	UPDATE accounts set current_balance = current_balance + 100
		WHERE current_balance < 5000;
	# Check number of affected rows
	IF @@ROWCOUNT > 200 
		BEGIN 
        	# Rollback the transaction
			ROLLBACK TRAN; 
			SELECT 'More accounts than expected. Rolling back'; 
		END
	ELSE
		BEGIN 
        	# Commit the transaction
			COMMIT TRAN; 
			SELECT 'Updates commited'; 
		END

# @@TRANCOUNT and savepoint 

In [ ]:
BEGIN TRY
	# Begin the transaction
	BEGIN TRAN;
    	# Correct the mistake
		UPDATE accounts SET current_balance = current_balance + 200
			WHERE account_id = 10;
    	# Check if there is a transaction
		IF @@TRANCOUNT > 0     
    		# Commit the transaction
			COMMIT TRAN;
     
	SELECT * FROM accounts
    	WHERE account_id = 10;      
END TRY
BEGIN CATCH  
    SELECT 'Rolling back the transaction'; 
    # Check if there is a transaction
    IF @@TRANCOUNT > 0   	
    	# Rollback the transaction
        ROLLBACK TRAN;
END CATCH

In [ ]:
BEGIN TRAN;
	# Mark savepoint1
	SAVE TRAN savepoint1;
	INSERT INTO customers VALUES ('Mark', 'Davis', 'markdavis@mail.com', '555909090');

	# Mark savepoint2
    SAVE TRAN savepoint2;
	INSERT INTO customers VALUES ('Zack', 'Roberts', 'zackroberts@mail.com', '555919191');

	# Rollback savepoint2
	ROLLBACK TRAN savepoint2;
    # Rollback savepoint1
	ROLLBACK TRAN savepoint1;

	# Mark savepoint3
	SAVE TRAN savepoint3;
	INSERT INTO customers VALUES ('Jeremy', 'Johnsson', 'jeremyjohnsson@mail.com', '555929292');
# Commit the transaction
COMMIT TRAN;

# XACT_ABORT & XACT_STATE
 

In [ ]:
# If there is an error and XACT_ABORT is set to ON, the transaction will always be rollbacked.

In [ ]:
# Use the appropriate setting
SET XACT_ABORT ON;
# Begin the transaction
BEGIN TRAN; 
	UPDATE accounts set current_balance = current_balance - current_balance * 0.01 / 100
		WHERE current_balance > 5000000;
	IF @@ROWCOUNT <= 10	
    	# Throw the error
		THROW 55000, 'Not enough wealthy customers!', 1;
	ELSE		
    	# Commit the transaction
		COMMIT TRAN; 

In [ ]:
# Use the appropriate setting
SET XACT_ABORT ON;
BEGIN TRY
	BEGIN TRAN;
		INSERT INTO customers VALUES ('Mark', 'Davis', 'markdavis@mail.com', '555909090');
		INSERT INTO customers VALUES ('Dylan', 'Smith', 'dylansmith@mail.com', '555888999');
	COMMIT TRAN;
END TRY
BEGIN CATCH
	# Check if there is an open transaction
	IF XACT_STATE() <> 0
    	# Rollback the transaction
		ROLLBACK TRAN;
    #Select the message of the error
    SELECT ERROR_MESSAGE() AS Error_message;
END CATCH

# Transaction isolation levels
 

In [ ]:
# Which of the following is true about these concurrency phenomena?

# Phantom reads occur when a transaction reads some records twice, but the first result it gets is different 
# from the second result as a consequence of another committed transaction having inserted a row.

In [ ]:
BEGIN TRAN

  INSERT INTO customers (first_name, last_name, email, phone)
  VALUES ('Ann', 'Ros', 'aros@mail.com', '555555555')

  DECLARE @cust_id INT = scope_identity()

  INSERT INTO accounts (account_number, customer_id, current_balance)
  VALUES ('55555555555010121212', @cust_id, 150)

COMMIT TRAN

In [ ]:
# Set the appropriate isolation level
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED

	# Select first_name, last_name, email and phone
	SELECT
    	first_name, 
        last_name, 
        email,
        phone
    FROM customers;

# READ COMMITTED & REPEATABLE READ
 

In [ ]:
# REPEATABLE READ isolation level can also prevent dirty reads.

In [ ]:
# Set the appropriate isolation level
SET TRANSACTION ISOLATION LEVEL READ COMMITTED

# Count the accounts
SELECT COUNT(*) AS number_of_accounts
FROM accounts
WHERE current_balance >= 50000;

# SERIALIZABLE isolation level

In [ ]:
# Set the appropriate isolation level
SET transaction ISOLATION LEVEL SERIALIZABLE

# Begin a transaction
BEGIN TRAN

SELECT * FROM customers;

# After some mathematical operations, we selected information from the customers table.
SELECT * FROM customers;

# Commit the transaction
COMMIT TRAN

In [ ]:
# Set the appropriate isolation level
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE

# Begin a transaction
BEGIN TRAN

# Select customer_id between 1 and 10
SELECT * 
FROM customers
WHERE customer_id BETWEEN 1 AND 10;

# After completing some mathematical operation, select customer_id between 1 and 10
SELECT * 
FROM customers
WHERE customer_id BETWEEN 1 AND 10;

# Commit the transaction
COMMIT TRAN

# SNAPSHOT

In [ ]:
ALTER DATABASE myDatabaseName SET ALLOW_SNAPSHOT_ISOLATION ON 
ALTER DATABASE myDatabaseName SET READ_COMMITTED_SNAPSHOT ON.

In [ ]:
The WITH (NOLOCK) option behaves like the READ UNCOMMITTED isolation level. 
But whereas the isolation level applies for the entire connection, WITH NOLOCK applies to a specific table.